In [6]:
import pandas as pd
import pandas as pd
import numpy as np

df = pd.read_excel('Cryotherapy.xlsx')
print("Dataframe info before the binning ",df.shape)
print("Dataframe info before the binning \n",df)

Dataframe info before the binning  (90, 7)
Dataframe info before the binning 
     sex  age   Time  Number_of_Warts  Type  Area  Result_of_Treatment
0     1   35  12.00                5     1   100                    0
1     1   29   7.00                5     1    96                    1
2     1   50   8.00                1     3   132                    0
3     1   32  11.75                7     3   750                    0
4     1   67   9.25                1     1    42                    0
..  ...  ...    ...              ...   ...   ...                  ...
85    2   34  12.00                3     3    95                    0
86    2   20   3.50                6     1    75                    1
87    2   35   8.25                8     3   100                    0
88    1   24  10.75               10     1    20                    1
89    1   19   8.00                8     1   160                    1

[90 rows x 7 columns]


In [7]:
# Bin the data frame by "age" with 4 bins...
age_bins = np.linspace(df.age.min(), df.age.max(), 4)
#print("Age bin",age_bins)
labels = [1,2,3]
df['agebinned'] = pd.cut(df['age'], bins=age_bins, labels=labels,include_lowest= True)
#print(df['agebinned'])

# Bin the data frame by "Time" with 4 bins...
time_bins = np.linspace(df.Time.min(), df.Time.max(), 4)
#print(time_bins)
labels = [1,2,3]
df['timebinned'] = pd.cut(df['Time'], bins=time_bins, labels=labels,include_lowest= True)
#print(df['timebinned'])

# Bin the data frame by "Number_of_Warts" with 4 bins...
wart_bins = np.linspace(df.Number_of_Warts.min(), df.Number_of_Warts.max(), 4)
#print(wart_bins)
labels = [1,2,3]
df['wartbinned'] = pd.cut(df['Number_of_Warts'], bins=wart_bins, labels=labels,include_lowest= True)
#print(df['wartbinned'])

# Bin the data frame by "Area" with 4 bins...
area_bins = np.linspace(df.Area.min(), df.Area.max(), 4)
#print(area_bins)
labels = [1,2,3]
df['areabinned'] = pd.cut(df['Area'], bins=area_bins, labels=labels,include_lowest= True)
#print(df['areabinned'])


#drop original columns by name
df.drop(['Number_of_Warts', 'Area','Time','age'], axis=1, inplace=True)

print("Dataframe shape after binning and removing the original column",df.shape)
print("Dataframe info after binning and removing the original column \n",df)

Dataframe shape after binning and removing the original column (90, 7)
Dataframe info after binning and removing the original column 
     sex  Type  Result_of_Treatment agebinned timebinned wartbinned areabinned
0     1     1                    0         2          3          2          1
1     1     1                    1         1          2          2          1
2     1     3                    0         3          2          1          1
3     1     3                    0         1          3          2          3
4     1     1                    0         3          3          1          1
..  ...   ...                  ...       ...        ...        ...        ...
85    2     3                    0         2          3          1          1
86    2     1                    1         1          1          2          1
87    2     3                    0         2          3          2          1
88    1     1                    1         1          3          3          1
89    1

In [8]:
from pomegranate import *

model = BayesianNetwork.from_samples(df.to_numpy(), state_names=df.columns.values, algorithm='exact')
#plt.plot()
model.sample()

array([[2, 2, 1, 1, 1, 3, 1]])

# Column order after binning is - sex, Type, Result_of_Treatment, agebinned, timebinned, wartbinned, areabinned

In [9]:
model.bake()

In [10]:
model.predict_proba({})

array([{
           "class" : "Distribution",
           "dtype" : "int",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "1" : 0.5222222222222223,
                   "2" : 0.4777777777777778
               }
           ],
           "frozen" : false
       }                                    ,
       {
           "class" : "Distribution",
           "dtype" : "int",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "1" : 0.5999999999999999,
                   "2" : 0.10000000000000012,
                   "3" : 0.3000000000000001
               }
           ],
           "frozen" : false
       }                                     ,
       {
           "class" : "Distribution",
           "dtype" : "int",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "0" : 0.46666666666666656,
                   "1" : 0.533

In [11]:
def getbinnedValue(val,binname):
    binvalue=(pd.cut([val], bins=binname, labels=labels,include_lowest=True))
    return binvalue[0]

In [12]:
#Predict whether the treatment was successful or not using the following data:
#Female Patient who is 33 years old with 12 number of type 3 warts.

#Sex=1, considered as female

#sex=1
#age=33 > 2 bin - 32.33333333< x<=49.66666667 67
#type=3
#warts=12 > 3 bin   1<X<=4.66666667<X<=8.33333333<X<=12

result_prob_mod=model.predict_proba([[1, 3, None, getbinnedValue(33,age_bins), None, getbinnedValue(12,wart_bins), None]])

print("Predict whether the treatment was successful or not - Female Patient who is 33 years old with 12 number of type 3 warts")
print("Probablity of Unsuccess",((((result_prob_mod[0])[2]).parameters)[0])[0])
print("Probablity of Success",((((result_prob_mod[0])[2]).parameters)[0])[1])


Predict whether the treatment was successful or not - Female Patient who is 33 years old with 12 number of type 3 warts
Probablity of Unsuccess 0.9546044098573278
Probablity of Success 0.045395590142672186


In [13]:
#Sex=1, considered as female
#Infer the probability for the data with the following properties
#A male aged 67 who has type 1 wart in area 51 and had a successful treatment

#sex=2
#age=67 > 3 bin - 32.33333333< x<=49.66666667 67
#type=1
#warts=12 > 3 bin   1<X<=4.66666667<X<=8.33333333<X<=12
#area - 51 > 1 bin  4.         252.66666667 501.33333333 750.

print("Infer the probability for the data - A male aged 67 who has type 1 wart in area 51 and had a successful treatment")
print("Probablity is ",model.probability([[2, 1, 1, getbinnedValue(67,age_bins), None, getbinnedValue(12,wart_bins), getbinnedValue(51,area_bins)]]))

Infer the probability for the data - A male aged 67 who has type 1 wart in area 51 and had a successful treatment
Probablity is  0.0


In [14]:
#Find the probability of the patient Age 15, Gender Male, Number of warts 5, Type 2, Area 100 and Treatment unsuccessful.
#model.predict_proba([[sex, Type, Result, agebinned, timebinned, wartbinned, areabinned]])
#Sex=1, considered as female

#sex=2
#age=15 > 1 bin - 15<X<32.33333333
#type=2
#warts=5 > 2 bin - 4.66666667<X<8.33333333
#Area 100 > 1 bin - 4<X<252.66666667

print("Find the probability of the patient - Age 15, Gender Male, Number of warts 5, Type 2, Area 100 and Treatment unsuccessful")
print("Probablity is ",model.probability([[2, 2, 0, getbinnedValue(15,age_bins), None, getbinnedValue(5,wart_bins), getbinnedValue(100,area_bins)]]))


Find the probability of the patient - Age 15, Gender Male, Number of warts 5, Type 2, Area 100 and Treatment unsuccessful
Probablity is  0.0038479915775834145
